In [16]:
from datasets import load_dataset

In [18]:
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

Using the latest cached version of the dataset since rotten_tomatoes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/dhruv/.cache/huggingface/datasets/rotten_tomatoes/default/0.0.0/aa13bc287fa6fcab6daf52f0dfb9994269ffea28 (last modified on Mon Apr 22 16:51:05 2024).


In [19]:
for item in test_dataset:
    if item['label'] == 1:
        print(item['text'])
        print()

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .

consistently clever and suspenseful .

it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .

the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .

red dragon " never cuts corners .

fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .

throws in enough clever and unexpected twists to make the formula feel fresh .

weighty and ponderous but every bit as filling as the treat of the title .

a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .

generates an enormous feeling of empathy for it

### Use BERT to have single vector for sentence

In [35]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
import torch
# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [36]:
# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [45]:
def get_doc_embedding(document, tokenizer, model):
    # Tokenize input text
    inputs = tokenizer(document, return_tensors="pt", padding=True, truncation=True)
    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**inputs)
    # Extract document embedding from hf model output
    document_embedding = outputs.last_hidden_state.mean(dim=1).squeeze(0)
    return document_embedding.numpy()

# def get_doc_embedding(document, model):
#     return model.encode(document)

In [46]:
# Example document from GLUE SST dataset
document = "those so-so films that could have been much better"
doc_emb = get_doc_embedding(document, tokenizer, model)
print(doc_emb.shape)


(768,)


### Save the vectors in pkl files

In [47]:
import tqdm
import numpy as np
import pickle

In [55]:
doc_vectors = [] 
# Init dummylist to decode whihc split train/val/test
dummy_list = test_dataset

In [56]:
for item in tqdm.tqdm(dummy_list):
    sentence = item['text']
    doc_emb = get_doc_embedding(sentence, tokenizer, model)
    doc_vectors.append(doc_emb)

100%|██████████| 1066/1066 [00:38<00:00, 27.51it/s]


In [57]:
# Open the file and store the reduced sentence vectors
with open('rotten-test-768.pkl', 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump(doc_vectors, file)

### Use PCA to reduce vector dimensions

In [37]:
# doc_vectors_np = np.array(doc_vectors)

# # 768 is emb dimension
# doc_vec_rehsaped = doc_vectors_np.reshape((len(dummy_list),768))

# # Apply PCA for dimensionality reduction
# pca = PCA(n_components = target_dimension)
# pca.fit(doc_vec_rehsaped)

# final_vecs = pca.transform(doc_vec_rehsaped)